In [1]:
from glob import glob
import os
from pathlib import Path
import shutil
import urllib.request as request
from contextlib import closing

import matplotlib.pyplot as plt
import pygrib
import xarray as xr
import numpy as np
import cartopy.feature as cfeature
from cartopy import crs as ccrs
from metpy.plots import USCOUNTIES, ctables
#import cf2cdm
from datetime import datetime
from dateutil import tz

In [2]:
grib_dir = Path('.') / '..' / '..' / 'grib'

In [15]:
# Download files

date = 20200709
run = '18'
fhr = '07'
left_lon = 360-90
right_lon = 360-77
top_lat = 36
bottom_lat = 24

nomads_url = ('https://nomads.ncep.noaa.gov/cgi-bin/filter_hrefconus.pl?'
              f'file=href.t{run}z.conus.prob.f{fhr}.grib2'
              '&all_lev=on&var_MAXREF=on&subregion='
              f'&leftlon={left_lon}&rightlon={right_lon}&toplat={top_lat}&bottomlat={bottom_lat}&'
              f'dir=/href.{date}/ensprod')

def download_gribfilter(file_url):
    with closing(request.urlopen(file_url)) as r:
        with open(f'{grib_dir}/href/href.t{run}z.conus.prob.maxrefl.f{fhr}.grib2', 'wb') as f:
            shutil.copyfileobj(r, f)
        
download_gribfilter(nomads_url)

In [16]:
#%%capture
# Read in files and acquire data needed for plotting
ds = pygrib.open(f'{grib_dir}/href/href.t{run}z.conus.prob.maxrefl.f{fhr}.grib2',)

# Reset the grib messages to 0 so you're at beginning of the file
ds.seek(0)

# Get grib message
# For reference, here are the corresponding grib messages with reflectivity exceedance
# 1 = 40dbZ; 2 = 50dbZ,
ds = ds[1]

# Extract lats, lons, and data values
lats, lons = ds.latlons()
href_values = ds.values

In [17]:
def get_gridpoint_value(lat_of_interest: float, lon_of_interest: float) -> int:
    '''
    Calculate the closest gridpoint value to a given latitude
    and longitude.
    '''
    abs_lat = np.abs(lats - lat_of_interest)
    abs_lon = np.abs(lons - lon_of_interest)
    
    overlap_latlon = np.maximum(abs_lon, abs_lat)
    latlon_idx = np.argmin(overlap_latlon)
    
    href_1d = href_values.flatten()
    
    return href_1d[latlon_idx]


prob = get_gridpoint_value(29.65, -82.32)
print(prob)

75.5
